In [1]:
import os
import sys

parent_dir = os.path.abspath("/home/leandre/Projects/BioMoQA_Playground")

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.data_pipeline.ipbes.create_ipbes_raw import loading_pipeline_from_raw

In [2]:
datasets = loading_pipeline_from_raw(multi_label=True)

2025-07-11 16:21:27,194 - INFO - load raw positive datasets
2025-07-11 16:21:27,195 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-11 16:21:27,196 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-11 16:21:27,779 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-11 16:21:27,779 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-11 16:21:27,780 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-11 16:21:28,015 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-11 16:21:28,016 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-11 16:21:28,017 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-11 16:21:28,273 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-11 16:21:28,274 - INFO - pos_ds features for IAS : {'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None), 'Publication Year': Value(dtype='int32', id=None)}
2025-07-11 16:21:28,274 - INFO - load raw negative dataset
2025-07-11 16:21:28,275 - INFO - creating corpus dataset


Resolving data files:   0%|          | 0/2524 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

2025-07-11 16:21:55,266 - INFO - dataset loaded
2025-07-11 16:21:55,267 - INFO - ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
2025-07-11 16:21:55,267 - INFO - renaming raw positive datasets
2025-07-11 16:21:55,281 - INFO - Finished positives and negatives creation pipeline


In [3]:
datasets

([Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 5251
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 6281
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 3109
  })],
 Dataset({
     features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
     num_rows: 4227131
 }))

In [5]:
pos_ds_list=datasets[0]
neg_ds=datasets[1]

In [6]:
clean_pos=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is None and batch['Item Type'][i] == 'journalArticle' and (batch['title'][i] is not None or batch['abstract'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

Filter (num_proc=20):   0%|          | 0/5251 [00:00<?, ? examples/s]

In [9]:
print(clean_pos)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 328
})


In [14]:
clean_pos_a=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is not None and batch['Item Type'][i] == 'journalArticle' for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

In [15]:
print(clean_pos_a)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 3829
})


In [17]:
clean_pos_b=clean_pos_a.filter(lambda batch : [(batch['abstract'][i] is not None and batch['title'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(clean_pos_b)

Filter (num_proc=20):   0%|          | 0/3829 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 2253
})


In [ ]:
none_abs=clean_pos_a.filter(lambda batch : [batch['abstract'][i] is None or batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 1576
})


In [21]:
print(none_abs[2])

{'doi': 'https://doi.org/10.1890/02-5002', 'title': 'Fire and grazing impacts on plant diversity and alien plant invasions in the southern Sierra Nevada', 'abstract': None, 'Item Type': 'journalArticle', 'Publication Year': 2003}


In [10]:
none_abs_instance=corpus_ds.filter(lambda batch : [batch['display_name'][i] == 'Germination and early growth of Ailanthus< and tulip poplar in three levels of forest disturbance' if batch['display_name'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [11]:
pos_doi_set=set(pos_ds_list[0]['doi'])
pos_doi_set.remove(None)
none_abs_instance=corpus_ds.filter(lambda batch : [any(batch['doi'][i].endswith(p_doi) for p_doi in pos_doi_set) if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=30)

In [12]:
print(len(pos_doi_set))

3861


In [13]:
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 941
})


In [14]:
print(corpus_ds['doi'][0:10])

['https://doi.org/10.1016/s0016-0032(29)90208-8', 'https://doi.org/10.1016/s0016-0032(29)90638-4', 'https://doi.org/10.1098/rstl.1829.0026', 'https://doi.org/10.1016/s0016-0032(29)90698-0', 'https://doi.org/10.1080/14786442908675166', 'https://doi.org/10.5962/bhl.title.109375', 'https://doi.org/10.1016/s0016-0032(29)90539-1', 'https://doi.org/10.1016/s0016-0032(29)90045-4', 'https://doi.org/10.1016/s0016-0032(29)90465-8', 'https://doi.org/10.1098/rstl.1816.0012']


In [15]:
print(corpus_ds.filter(lambda batch : [batch['id'][i].endswith('W2142524821') if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [16]:
print(corpus_ds['id'][0])

https://openalex.org/W1979751022


In [18]:
recent_pos=clean_pos_b.filter(lambda batch : [batch["Publication Year"][i]>=2021 if batch["Publication Year"][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(recent_pos)

Filter (num_proc=20):   0%|          | 0/2253 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 158
})


In [22]:
from src.data_pipeline.ipbes.preprocess_ipbes import data_pipeline

clean_ds=data_pipeline(multi_label=True)
print(clean_ds)
print(clean_ds.features)

2025-07-08 11:11:48,185 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-08 11:11:48,186 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-08 11:11:48,592 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-08 11:11:48,593 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-08 11:11:48,595 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-08 11:11:48,751 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-08 11:11:48,752 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-08 11:11:48,753 - INFO - 128


...

Map (num_proc=4):   0%|          | 0/4197513 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4197513 [00:00<?, ? examples/s]

Size of the dataset before cleaning: 4197513
Applying clean_filter...


Filter (num_proc=32):   0%|          | 0/4197513 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 4111303
})
{'doi': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'abstract': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None), 'Publication Year': Value(dtype='float64', id=None), 'IAS': Value(dtype='int64', id=None), 'SUA': Value(dtype='int64', id=None), 'VA': Value(dtype='int64', id=None)}


In [ ]:

print(clean_ds.filter(lambda batch : [batch['doi'][i] is None and (batch['IAS']==1 or batch['SUA']==1 or batch['VA']==1 ) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['abstract'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.to_pandas()['Item Type'].unique())



Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 747444
})
Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})


Filter (num_proc=20):   0%|          | 0/4111303 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})


KeyboardInterrupt: 